In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os


In [2]:
def preprocess_dataset(folderpath, dest_path, limit:int=None):
  folder_files = os.listdir(folderpath)

  for i, file in enumerate(folder_files, start=1):
    base_image = cv2.imread(f'{folderpath}/{file}')
    base_image = cv2.resize(base_image, (224,224))

    enhanced_image = enhance_contrast(base_image, False)
    max_pixel_diff, enhanced_green, g = enhance_green_channel(image=enhanced_image,
                                        show_channels=False,
                                        show_enhanced=False)

    blood_vessel_mask = extract_blood_vessels(image=enhanced_green,
                                            show_extracted=False)

    # Inside preprocess_dataset function, when calling extract_outliers
    outlier_mask = extract_outliers(image=g, g=g, max_pixel_diff=max_pixel_diff, show_extracted=False)


    overlap_mask = cv2.bitwise_and(blood_vessel_mask, outlier_mask)
    outlier_mask = np.subtract(outlier_mask, overlap_mask)
    outlier_mask[outlier_mask == 50] = 225

    merged_mask = cv2.bitwise_or(blood_vessel_mask, outlier_mask)
    merged_mask[merged_mask == 0] = 100
    
    rgb_mask = cv2.cvtColor(merged_mask,cv2.COLOR_GRAY2RGB)
    result = np.hstack((base_image, rgb_mask))
    cv2.imshow("output", result)

    final_output = cv2.resize(rgb_mask, (512,512))
    cv2.imwrite(f"{dest_path}/{file}", final_output) 

    if limit == None:
        pass

    elif i >= limit:
        break

def enhance_contrast(image:np.ndarray, show_enchanced:bool):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB) #convert rgb image to lab channels
    L, a, b = cv2.split(lab) #split lab channels

    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8)) #create Contrast Limited Adaptive Histogram Equalization (CLAHE)
    cL = clahe.apply(L) #apply to lab channel to increase contrast

    new_img = cv2.merge((cL,a,b)) #create new lab enhanced image
    enhanced_img = cv2.cvtColor(new_img, cv2.COLOR_LAB2BGR) #convert from lab to rgb

    if show_enchanced:
      result = np.hstack((image, enhanced_img))
      cv2.imshow("enhanced image", result) #update to local version @weixuan

    return enhanced_img


def enhance_green_channel(image:np.ndarray, kernel_size:tuple=(75,75), show_channels:bool=False, show_enhanced:bool=False):
    b,g,r = cv2.split(image)
    if show_channels:
      cv2.imshow("blue channel", b)
      cv2.imshow("green channel", g)
      cv2.imshow("red channel", r)

    kernel = np.ones(kernel_size, np.uint8)
    opening = cv2.morphologyEx(g, cv2.MORPH_OPEN, kernel)
    morph_green = cv2.subtract(g, opening)

    max_pixel_diff = np.max(morph_green)

    min_intensity = np.min(morph_green)
    max_intensity = np.max(morph_green)

    enhanced_green = ((morph_green - min_intensity) / (max_intensity - min_intensity)) * 255
    enhanced_green = enhanced_green.astype(np.uint8)

    if show_enhanced:
      result = np.hstack((g, enhanced_green))
      cv2.imshow("enhanced green channel", result)

    return max_pixel_diff, enhanced_green, g


def extract_blood_vessels(image:np.ndarray, block_size:int=31, calculated_mean:int=21, area_limit:int=3, show_extracted:bool=False):
    if block_size % 2 ==0:
      raise Exception("block_size must be an odd integer")

    while True:
      adaptive_threshold = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, block_size, calculated_mean)
      contours, _ = cv2.findContours(adaptive_threshold, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

      contours = sorted(contours, key=cv2.contourArea, reverse=True)
      contours = [cnt for cnt in contours if cv2.contourArea(cnt) > area_limit]
      max_contour_area = max([cv2.contourArea(cnt) for cnt in contours])
      if max_contour_area >= 10000:
        block_size += 2
        calculated_mean += 3
      
      else:
        break

    mask = np.zeros_like(image)
    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)

    gray_mask = np.copy(mask)
    gray_mask[mask == 255] = 50

    if show_extracted:
      result = np.hstack((image, gray_mask))
      cv2.imshow("vein extraction", result)

    return gray_mask


def extract_outliers(image: np.ndarray, g: np.ndarray, max_pixel_diff: int = 125, show_extracted: bool = False):
    # Initialize variables to None
    binary = binary2 = binary3 = None

    # Your existing conditional logic here...
    if max_pixel_diff < 165:
        _, binary = cv2.threshold(image, max_pixel_diff, 255, cv2.THRESH_BINARY)
        _, binary2 = cv2.threshold(image, max_pixel_diff, 255, cv2.THRESH_BINARY)
        _, binary3 = cv2.threshold(image, max_pixel_diff * 1.1, 255, cv2.THRESH_BINARY)

    elif 165<= max_pixel_diff <200:
      _, binary = cv2.threshold(image, max_pixel_diff*0.8, 255, cv2.THRESH_BINARY)
      _, binary2 = cv2.threshold(image, max_pixel_diff*0.85, 255, cv2.THRESH_BINARY)
      _, binary3 = cv2.threshold(image, max_pixel_diff*1.1, 255, cv2.THRESH_BINARY)

    elif 200<= max_pixel_diff <215:
      _, binary = cv2.threshold(image, max_pixel_diff*0.7, 255, cv2.THRESH_BINARY)
      _, binary2 = cv2.threshold(image, max_pixel_diff*0.75, 255, cv2.THRESH_BINARY)
      _, binary3 = cv2.threshold(image, max_pixel_diff*0.8, 255, cv2.THRESH_BINARY)

    elif 215 <= max_pixel_diff < 230:
      _, binary = cv2.threshold(image, max_pixel_diff*0.75, 255, cv2.THRESH_BINARY)
      _, binary2 = cv2.threshold(image, max_pixel_diff*0.85, 255, cv2.THRESH_BINARY)
      _, binary3 = cv2.threshold(image, max_pixel_diff*0.9, 255, cv2.THRESH_BINARY)

    elif 230<= max_pixel_diff <245:
      _, binary = cv2.threshold(image, max_pixel_diff*0.45, 255, cv2.THRESH_BINARY)
      _, binary2 = cv2.threshold(image, max_pixel_diff*0.5, 255, cv2.THRESH_BINARY)
      _, binary3 = cv2.threshold(image, max_pixel_diff, 255, cv2.THRESH_BINARY)

    elif max_pixel_diff >245:
      _, binary = cv2.threshold(image, max_pixel_diff*0.7, 255, cv2.THRESH_BINARY)
      _, binary2 = cv2.threshold(image, max_pixel_diff*0.75, 255, cv2.THRESH_BINARY)
      _, binary3 = cv2.threshold(image, max_pixel_diff, 255, cv2.THRESH_BINARY)



    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours2, _ = cv2.findContours(binary2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours3, _ = cv2.findContours(binary3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(g)
    mask2 = np.zeros_like(g)
    mask3 = np.zeros_like(g)

    cv2.drawContours(mask, contours, -1, (255), thickness=cv2.FILLED)
    cv2.drawContours(mask2, contours2, -1, (255), thickness=cv2.FILLED)
    cv2.drawContours(mask3, contours3, -1, (255), thickness=cv2.FILLED)

    merge_mask = np.subtract(mask, mask2)
    merge_mask = np.add(merge_mask, mask3)

    gray_mask = np.copy(merge_mask)
    gray_mask[mask == 255] = 50

    if show_extracted:
      result = np.hstack((image, gray_mask))
      cv2.imshow("outlier extraction", result)

    return gray_mask


In [ ]:
preprocess_dataset(r'E:\Diabetic_Retinopathy_Detection\input\grading_images\processed_images', r'E:\Diabetic_Retinopathy_Detection\input\grading_images\augment')


In [2]:
import os
import cv2
import numpy as np

# Base input and output folder paths
base_folder_path = r"D:\dr_images\EyePacs\train\4"
base_processed_folder_path = r"D:\dr_images\New folder"

# Define a list of acceptable file extensions
acceptable_extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.bmp']

def process_image(file_path, processed_file_path):
    """Process and save an image."""
    sample_img = cv2.imread(file_path)
    
    # Check if the image was loaded properly
    if sample_img is None:
        print(f"Failed to load image: {file_path}")
        return
    
    # Convert to RGB, apply processing, and save
    sample_img = cv2.cvtColor(sample_img, cv2.COLOR_BGR2RGB)
    ben_sample_img = cv2.addWeighted(sample_img, 4, cv2.GaussianBlur(sample_img, (0,0), 10), -4, 128)
    ben_sample_img_bgr = cv2.cvtColor(ben_sample_img, cv2.COLOR_RGB2BGR)
    
    cv2.imwrite(processed_file_path, ben_sample_img_bgr)
    print(f"Processed and saved: {processed_file_path}")

def replicate_and_process_directory_structure(source_folder, target_folder):
    """Replicate directory structure and process images."""
    for root, dirs, files in os.walk(source_folder):
        # Determine the path to the target directory
        relative_path = os.path.relpath(root, source_folder)
        target_path = os.path.join(target_folder, relative_path)
        
        # Ensure the target directory exists
        os.makedirs(target_path, exist_ok=True)
        
        # Process each file
        for file in files:
            if os.path.splitext(file)[1].lower() in acceptable_extensions:
                file_path = os.path.join(root, file)
                processed_file_path = os.path.join(target_path, file)
                process_image(file_path, processed_file_path)

# Ensure the base processed images directory exists
os.makedirs(base_processed_folder_path, exist_ok=True)
replicate_and_process_directory_structure(base_folder_path, base_processed_folder_path)


Processed and saved: D:\dr_images\New folder\.\1002_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10047_left.jpeg
Processed and saved: D:\dr_images\New folder\.\1008_left.jpeg
Processed and saved: D:\dr_images\New folder\.\1008_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10125_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10125_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10153_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10153_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10159_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10489_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10510_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10579_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10579_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10606_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10606_right.jpeg
Processed and saved: D:\dr_images\New folder\.\1081

KeyboardInterrupt: 

In [3]:
import os
import cv2
import numpy as np

# Base input and output folder paths
base_folder_path = r"D:\dr_images\EyePacs\train\4"
base_processed_folder_path = r"D:\dr_images\New folder"

# Define a list of acceptable file extensions
acceptable_extensions = ['.jpg', '.jpeg', '.png', '.tiff', '.bmp']

def process_image(file_path, processed_file_path):
    """Process and save an image."""
    sample_img = cv2.imread(file_path)
    
    # Check if the image was loaded properly
    if sample_img is None:
        print(f"Failed to load image: {file_path}")
        return
    
    # Convert to YUV
    sample_img_yuv = cv2.cvtColor(sample_img, cv2.COLOR_BGR2YUV)
    
    # Equalize the histogram of the Y channel
    sample_img_yuv[:,:,0] = cv2.equalizeHist(sample_img_yuv[:,:,0])
    
    # Convert back to RGB
    sample_img_equalized = cv2.cvtColor(sample_img_yuv, cv2.COLOR_YUV2RGB)
    
    # Apply further processing
    ben_sample_img = cv2.addWeighted(sample_img_equalized, 4, cv2.GaussianBlur(sample_img_equalized, (0,0), 10), -4, 128)
    
    # Convert back to BGR for saving
    ben_sample_img_bgr = cv2.cvtColor(ben_sample_img, cv2.COLOR_RGB2BGR)
    
    cv2.imwrite(processed_file_path, ben_sample_img_bgr)
    print(f"Processed and saved: {processed_file_path}")

def replicate_and_process_directory_structure(source_folder, target_folder):
    """Replicate directory structure and process images."""
    for root, dirs, files in os.walk(source_folder):
        # Determine the path to the target directory
        relative_path = os.path.relpath(root, source_folder)
        target_path = os.path.join(target_folder, relative_path)
        
        # Ensure the target directory exists
        os.makedirs(target_path, exist_ok=True)
        
        # Process each file
        for file in files:
            if os.path.splitext(file)[1].lower() in acceptable_extensions:
                file_path = os.path.join(root, file)
                processed_file_path = os.path.join(target_path, file)
                process_image(file_path, processed_file_path)

# Ensure the base processed images directory exists
os.makedirs(base_processed_folder_path, exist_ok=True)
replicate_and_process_directory_structure(base_folder_path, base_processed_folder_path)


Processed and saved: D:\dr_images\New folder\.\10017_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10017_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10047_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10193_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10312_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10321_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10570_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10570_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10653_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10653_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10785_left.jpeg
Processed and saved: D:\dr_images\New folder\.\10785_right.jpeg
Processed and saved: D:\dr_images\New folder\.\1084_left.jpeg
Processed and saved: D:\dr_images\New folder\.\1084_right.jpeg
Processed and saved: D:\dr_images\New folder\.\10904_left.jpeg
Processed and saved: D:\dr_images\New folder\.\109

KeyboardInterrupt: 

In [5]:
import cv2
import numpy as np
import os
import time

def create_directory(directory):
    """Creates a new folder in the specified directory if the folder doesn't exist."""
    if not os.path.exists(directory):
        os.makedirs(directory)

def scale_radius(img, scale):
    """Scales an image to a given radius."""
    x = img[img.shape[0] // 2,:,:].sum(1)
    r = (x > x.mean() / 10).sum() / 2
    s = (scale * 1.0) / r
    return cv2.resize(img, (0, 0), fx=s, fy=s)

def process_image(img_path, processed_file_path, scale):
    """Processes an image by scaling, color balancing, and removing boundary effects."""
    img = cv2.imread(img_path)
    if img is None:
        print(f"Failed to load image: {img_path}")
        return
    
    img = scale_radius(img, scale)
    img = cv2.addWeighted(img, 4, cv2.GaussianBlur(img, (0, 0), scale/30), -4, 128)
    
    # Remove outer 10%
    mask = np.zeros(img.shape)
    cv2.circle(mask, (img.shape[1] // 2, img.shape[0] // 2), int(scale * 0.9), (1, 1, 1), -1, 8, 0)
    img = img * mask + 128 * (1 - mask)
    
    cv2.imwrite(processed_file_path, img)

def process_images_in_directory(source_folder, target_folder, scale):
    """Processes all images in a directory and saves them to a new location."""
    start_time = time.time()
    create_directory(target_folder)
    
    for file_name in os.listdir(source_folder):
        if file_name.lower().endswith(('.jpg', '.jpeg', '.png', '.tiff', '.bmp')):
            file_path = os.path.join(source_folder, file_name)
            processed_file_path = os.path.join(target_folder, file_name)
            process_image(file_path, processed_file_path, scale)
    
    print(f"--- {time.time() - start_time} seconds ---")

# Example usage
input_folder_path = r"H:\Diabetic_Retinopathy_Detection\input\grading_images\data"
output_folder_path = r"H:\Diabetic_Retinopathy_Detection\input\grading_images\process_img"
scale = 500
process_images_in_directory(input_folder_path, output_folder_path, scale)


--- 0.0 seconds ---
